In [1]:
import sqlite3
import re
import pandas as pd

def write_to_file(unique_list):
    with open('ingredients_with_count.csv', 'w') as f:
        for item in range(len(unique_list)):
            f.write("\""+unique_list[item] +"\","+str(count[item])+ "\n")


conn = sqlite3.connect('db1.db')
c = conn.cursor()
c.execute('''SELECT ingredients from hack_ru''')

In [2]:
a = c.fetchall()

i = 0
unique_list = list()
count = list()
ingredients_per_entry = list()

for row in a:
    temp3 = re.sub("[\(\[].*?[\)\]]", "", str(row[0]))
    for temp in (temp3.split(',')):
        temp = temp.replace('ingredients', ',')
        temp = re.sub("[\(\[].*?[\)\]]", "", temp)
        temp = temp.replace('(', ',')
        temp = temp.replace(')', ',')
        # temp = temp.replace('/', ',')

        for ingred in temp.split(','):
            ingred = ingred.strip().lower()
            ingred = ingred.replace('active ingredient', '').lower()
            ingred = ingred.replace(':', '').strip().lower()
            if (ingred != ""):
                if (ingred not in unique_list):
                    unique_list.append(ingred)
                    count.append(1)
                else:
                    count[unique_list.index(ingred)] += 1


In [3]:
def remove_element(name):
    i=unique_list.index(name)
    del unique_list[i]
    del count[i]

In [1]:
# try:
#     for ele in range(len(unique_list)):
#         if(len(unique_list[ele])<3):
#             del unique_list[ele]
#             del count[ele]
#         if(len(unique_list[ele])<4):
#             if(count[ele]<5):
#                 del unique_list[ele]
#                 del count[ele]
# except:
#     pass


# try:
#     for i in range(len(unique_list)):
#         if(count[i]<=3):
#             del unique_list[i]
#             del count[i]
# except:
#     pass
# remove_element("bicarbonate")
# remove_element("sodium")
# remove_element("hydrated")
# remove_element("alum")
# remove_element("active")
# remove_element("sulphate")
# remove_element("dioxide")
# remove_element("ingredient")

In [5]:
for row in a:
    temp = list()
    for ingred in unique_list:
        if ingred in str(row[0]).lower():
            temp.append(ingred)
    ingredients_per_entry.append(temp[:])
    temp.clear()

In [7]:
ingredients_dataframe = pd.read_csv('ingredients_with_count.csv')
ingredients_dataframe = ingredients_dataframe.sort_values(by = ['count'], ascending = False)

In [8]:
finalDataFrame = pd.read_csv("hack_ru_test.csv")
print(finalDataFrame.info())
templist = []
for ingredient in unique_list:
    templist.clear()
    for product_ingredient in ingredients_per_entry:
        if ingredient in product_ingredient:
            templist.append(1)
        else:
            templist.append(0)
    finalDataFrame[str(ingredient)] = templist

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14741 entries, 0 to 14740
Data columns (total 6 columns):
field1                       14741 non-null int64
country                      14741 non-null object
company                      14741 non-null object
total_pack_size_ml_g         14741 non-null float64
unit_pack_size_ml_g          14741 non-null float64
price_per_100g_ml_dollars    14741 non-null float64
dtypes: float64(3), int64(1), object(2)
memory usage: 691.1+ KB
None


In [49]:
df=finalDataFrame
df=df.drop(columns="field1")
#df=df.iloc[:,0:900]
df=pd.get_dummies(df,sparse=True)
df=df.astype('float64')
df.head()



,total_pack_size_ml_g,unit_pack_size_ml_g,price_per_100g_ml_dollars,ci 77220,aqua,sorbitol,hydrated silica,sodium lauryl sulfate,flavor,cellulose gum,...,company_Brand 990,company_Brand 991,company_Brand 992,company_Brand 993,company_Brand 994,company_Brand 995,company_Brand 996,company_Brand 997,company_Brand 998,company_Brand 999
0,130.0,130.0,0.32,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,90.0,90.0,1.22,1.0,1.0,1.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,120.0,120.0,1.26,0.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100.0,100.0,4.92,0.0,1.0,1.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,160.0,160.0,0.79,0.0,1.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df.to_csv('one_hot_encoded.csv');